In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 1. 激活环境秘钥
load_dotenv(override=True)

# 2. 初始化大模型 (Temperature 设为 0，保证信息提取的绝对严谨，不瞎编)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 定义我们期望的数据结构 (Pydantic 模型)
# 这就相当于告诉 AI：“我只要这三个字段，别给我说废话”
class PaperExtraction(BaseModel):
    algorithm: str = Field(description="论文中提出或使用的核心算法/模型名称")
    dataset: str = Field(description="实验中使用的主要数据集名称")
    accuracy: float = Field(description="模型在测试集上达到的准确率，仅提取数字(浮点数)")

# 初始化解析器
parser = PydanticOutputParser(pydantic_object=PaperExtraction)

# 4. 构建包含 System 和 Human 的高级 Prompt
# {format_instructions} 会自动注入我们刚才定义的 Pydantic 格式要求
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个严谨的 AI 算法评审专家。请根据用户的摘要提取关键信息。\n\n{format_instructions}"),
    ("user", "请提取以下摘要中的信息：\n{abstract_text}")
])

# 5. LCEL 链式调用 (Prompt -> LLM -> Parser)
chain = prompt | llm | parser

# 6. 测试数据：一段模拟的学术论文摘要
dummy_abstract = """
在本文中，我们针对大语言模型的推理效率优化问题进行了深入研究。
我们提出了一种全新的动态剪枝算法（Dynamic Token Pruning, DTP）。
为了验证该算法的有效性，我们在标准的 SQuAD 2.0 问答数据集上进行了广泛的消融实验。
实验结果表明，DTP 算法在保持推理速度提升 40% 的同时，依然维持了高达 89.5% 的准确率。
"""

print("正在分析文献摘要并提取结构化数据...\n")

# 7. 执行提取
result = chain.invoke({
    "abstract_text": dummy_abstract,
    "format_instructions": parser.get_format_instructions() # 自动获取格式化指令
})

# 8. 见证奇迹：输出的结果是一个完美的 Python 对象！
print(f"提取出的对象类型: {type(result)}\n")
print(f"核心算法: {result.algorithm}")
print(f"使用数据集: {result.dataset}")
print(f"模型准确率: {result.accuracy} (类型: {type(result.accuracy)})")

正在分析文献摘要并提取结构化数据...

提取出的对象类型: <class '__main__.PaperExtraction'>

核心算法: Dynamic Token Pruning (DTP)
使用数据集: SQuAD 2.0
模型准确率: 89.5 (类型: <class 'float'>)
